In [4]:

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,2,3'
import random
random.seed(1)

from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms
import torch
import utils

from tqdm import tqdm
from PIL import Image
import os
import pandas as pd
import csv
os.sys.path.append('..')

from hessian import get_hessian_eigenvalues, get_hessian_eigenvalues_from_sample, compute_multi_db_distance_for_single_sample, compute_multi_db_curvature, get_cosine_similarity, get_jac_mine
from utils import WrapModel, get_normalize

transformer = transforms.Compose([
            transforms.Resize([224, 224]),
            transforms.ToTensor(),
        ])



class WrapModel(torch.nn.Module):
    def __init__(self, normalize, model):
        super(WrapModel, self).__init__()
        self.model = model
        self.normal = normalize

    def forward(self, input):
        return self.model(self.normal(input))
    

class ImageNette(torchvision.datasets.ImageFolder):
    def __init__(self, root, transform):
        super(ImageNette, self).__init__(root=root, transform=transform)
        self.transform = transform
        self.samples = random.sample(self.samples, 100)

    def __getitem__(self, index):
        path, target = self.samples[index]
        sample = Image.open(path).convert('RGB')
        if self.transform is not None:
            sample = self.transform(sample)
        if self.target_transform is not None:
            target = self.target_transform(target)
        return sample, target
    


X = ImageNette(root='../../Datasets/imagenette/test-val', transform=transformer)


from torch.utils.data import DataLoader


loader = DataLoader(X, batch_size=1, shuffle=False)





In [5]:
def get_model(arch, model_path):
    model = utils.get_arch(arch, 'imagenett')
    state_dict = torch.load(model_path, map_location=torch.device('cpu'))
    model.load_state_dict(state_dict['model_state_dict'])
    del state_dict
    normalize = utils.get_normalize('imagenet')
    model = WrapModel(model=model,normalize=normalize).cuda()
    model.eval()
    return model
  
def get_smoothness(arch, model_path):
    model = get_model(arch, model_path)
    sharpness_list = []
    for x, y in loader:
        x = x.cuda()
        y = y.cuda()
        sharpness_list.append( 
            abs(get_hessian_eigenvalues_from_sample(model, torch.nn.CrossEntropyLoss(), x, y, neigs=1, which='LA') ).item()
        )
    print("sample smoothness:", sum(sharpness_list) / len(sharpness_list) )
    return sum(sharpness_list) / len(sharpness_list)

In [6]:
aug_model_path0 = [
    './imagenett-models/resnet50-backup-lr-1/model-fin-model.pkl',
    './imagenett-models/resnet50-cutout-80-backup-lr-1/model-fin-model.pkl',
    './imagenett-models/resnet50-cutout-100-backup-lr-1/model-fin-model.pkl',
    './imagenett-models/resnet50-cutout-120-backup-lr-1/model-fin-model.pkl',
    './imagenett-models/resnet50-cutout-140-backup-lr-1/model-fin-model.pkl',
    './imagenett-models/resnet50-cutout-160-backup-lr-1/model-fin-model.pkl',
    './imagenett-models/resnet50-cutmix-0.1-backup-lr-1/model-fin-model.pkl',
    './imagenett-models/resnet50-cutmix-0.3-backup-lr-1/model-fin-model.pkl',
    './imagenett-models/resnet50-cutmix-0.5-backup-lr-1/model-fin-model.pkl',
    './imagenett-models/resnet50-cutmix-0.7-backup-lr-1/model-fin-model.pkl',
    './imagenett-models/resnet50-cutmix-0.9-backup-lr-1/model-fin-model.pkl',
    './imagenett-models/resnet50-mixup-0.1-backup-lr-1/model-fin-model.pkl',
    './imagenett-models/resnet50-mixup-0.3-backup-lr-1/model-fin-model.pkl',
    './imagenett-models/resnet50-mixup-0.5-backup-lr-1/model-fin-model.pkl',
    './imagenett-models/resnet50-mixup-0.7-backup-lr-1/model-fin-model.pkl',
    './imagenett-models/resnet50-mixup-0.9-backup-lr-1/model-fin-model.pkl',
    './imagenett-models/resnet50-label-smoothing-0.1-backup-lr-1/model-fin-model.pkl',
    './imagenett-models/resnet50-label-smoothing-0.2-backup-lr-1/model-fin-model.pkl',
    './imagenett-models/resnet50-label-smoothing-0.3-backup-lr-1/model-fin-model.pkl',
    './imagenett-models/resnet50-label-smoothing-0.4-backup-lr-1/model-fin-model.pkl',
    './imagenett-models/resnet50-label-smoothing-0.5-backup-lr-1/model-fin-model.pkl',
]

aug_model_path1 = [
    './imagenett-models/resnet50-backup-lr-1-seed-1/model-fin-model.pkl',
    './imagenett-models/resnet50-cutout-80-backup-lr-1-seed-1/model-fin-model.pkl',
    './imagenett-models/resnet50-cutout-100-backup-lr-1-seed-1/model-fin-model.pkl',
    './imagenett-models/resnet50-cutout-120-backup-lr-1-seed-1/model-fin-model.pkl',
    './imagenett-models/resnet50-cutout-140-backup-lr-1-seed-1/model-fin-model.pkl',
    './imagenett-models/resnet50-cutout-160-backup-lr-1-seed-1/model-fin-model.pkl',
    './imagenett-models/resnet50-cutmix-0.1-backup-lr-1-seed-1/model-fin-model.pkl',
    './imagenett-models/resnet50-cutmix-0.3-backup-lr-1-seed-1/model-fin-model.pkl',
    './imagenett-models/resnet50-cutmix-0.5-backup-lr-1-seed-1/model-fin-model.pkl',
    './imagenett-models/resnet50-cutmix-0.7-backup-lr-1-seed-1/model-fin-model.pkl',
    './imagenett-models/resnet50-cutmix-0.9-backup-lr-1-seed-1/model-fin-model.pkl',
    './imagenett-models/resnet50-mixup-0.1-backup-lr-1-seed-1/model-fin-model.pkl',
    './imagenett-models/resnet50-mixup-0.3-backup-lr-1-seed-1/model-fin-model.pkl',
    './imagenett-models/resnet50-mixup-0.5-backup-lr-1-seed-1/model-fin-model.pkl',
    './imagenett-models/resnet50-mixup-0.7-backup-lr-1-seed-1/model-fin-model.pkl',
    './imagenett-models/resnet50-mixup-0.9-backup-lr-1-seed-1/model-fin-model.pkl',
    './imagenett-models/resnet50-label-smoothing-0.1-backup-lr-1-seed-1/model-fin-model.pkl',
    './imagenett-models/resnet50-label-smoothing-0.2-backup-lr-1-seed-1/model-fin-model.pkl',
    './imagenett-models/resnet50-label-smoothing-0.3-backup-lr-1-seed-1/model-fin-model.pkl',
    './imagenett-models/resnet50-label-smoothing-0.4-backup-lr-1-seed-1/model-fin-model.pkl',
    './imagenett-models/resnet50-label-smoothing-0.5-backup-lr-1-seed-1/model-fin-model.pkl',
]

aug_model_path2 = [
    './imagenett-models/resnet50-backup-lr-1-seed-2/model-fin-model.pkl',
    './imagenett-models/resnet50-cutout-80-backup-lr-1-seed-2/model-fin-model.pkl',
    './imagenett-models/resnet50-cutout-100-backup-lr-1-seed-2/model-fin-model.pkl',
    './imagenett-models/resnet50-cutout-120-backup-lr-1-seed-2/model-fin-model.pkl',
    './imagenett-models/resnet50-cutout-140-backup-lr-1-seed-2/model-fin-model.pkl',
    './imagenett-models/resnet50-cutout-160-backup-lr-1-seed-2/model-fin-model.pkl',
    './imagenett-models/resnet50-cutmix-0.1-backup-lr-1-seed-2/model-fin-model.pkl',
    './imagenett-models/resnet50-cutmix-0.3-backup-lr-1-seed-2/model-fin-model.pkl',
    './imagenett-models/resnet50-cutmix-0.5-backup-lr-1-seed-2/model-fin-model.pkl',
    './imagenett-models/resnet50-cutmix-0.7-backup-lr-1-seed-2/model-fin-model.pkl',
    './imagenett-models/resnet50-cutmix-0.9-backup-lr-1-seed-2/model-fin-model.pkl',
    './imagenett-models/resnet50-mixup-0.1-backup-lr-1-seed-2/model-fin-model.pkl',
    './imagenett-models/resnet50-mixup-0.3-backup-lr-1-seed-2/model-fin-model.pkl',
    './imagenett-models/resnet50-mixup-0.5-backup-lr-1-seed-2/model-fin-model.pkl',
    './imagenett-models/resnet50-mixup-0.7-backup-lr-1-seed-2/model-fin-model.pkl',
    './imagenett-models/resnet50-mixup-0.9-backup-lr-1-seed-2/model-fin-model.pkl',
    './imagenett-models/resnet50-label-smoothing-0.1-backup-lr-1-seed-2/model-fin-model.pkl',
    './imagenett-models/resnet50-label-smoothing-0.2-backup-lr-1-seed-2/model-fin-model.pkl',
    './imagenett-models/resnet50-label-smoothing-0.3-backup-lr-1-seed-2/model-fin-model.pkl',
    './imagenett-models/resnet50-label-smoothing-0.4-backup-lr-1-seed-2/model-fin-model.pkl',
    './imagenett-models/resnet50-label-smoothing-0.5-backup-lr-1-seed-2/model-fin-model.pkl',
]

In [ ]:
aug_seed0=[]
for path in aug_model_path0:
    tmp0=get_smoothness('resnet50', path)
    aug_seed0.append(tmp0)
data0 = pd.DataFrame(aug_seed0)
data0.to_csv('4_2_MS_aug_seed0.csv')

aug_seed1=[]
for path in aug_model_path1:
    tmp1=get_smoothness('resnet50', path)
    aug_seed1.append(tmp1)
data1 = pd.DataFrame(aug_seed1)
data1.to_csv('4_2_MS_aug_seed1.csv')

aug_seed2=[]
for path in aug_model_path2:
    tmp2=get_smoothness('resnet50', path)
    aug_seed2.append(tmp2)
data2 = pd.DataFrame(aug_seed2)
data2.to_csv('4_2_MS_aug_seed2.csv')

sample smoothness: 3.4000890148000207
sample smoothness: 5.878831365732317
sample smoothness: 4.535730262473504
sample smoothness: 3.065003183931009
sample smoothness: 5.381168953345382
sample smoothness: 5.429407971917873
sample smoothness: 4.464308368638158
sample smoothness: 3.836712923704181
sample smoothness: 7.330816145772115
sample smoothness: 4.611068341191858
sample smoothness: 4.532924759835005
sample smoothness: 4.277770751426578
sample smoothness: 8.495881999929551
sample smoothness: 7.803900712244213
sample smoothness: 7.583729155976325
sample smoothness: 7.700261017680168
sample smoothness: 2.295670333430171
sample smoothness: 1.8719516809936614
sample smoothness: 1.4445061970967799
sample smoothness: 1.1833988637197763
sample smoothness: 0.5126268969569355
sample smoothness: 4.532514138455158
sample smoothness: 7.2015726732021985
sample smoothness: 4.972437803690069
sample smoothness: 7.830881709431723
sample smoothness: 5.331976346456338
sample smoothness: 6.64017584867

In [ ]:
gr_model_path0=[
    './imagenett-models/resnet50-mr-0.05-lr-1/model-fin-model.pkl',
    './imagenett-models/resnet50-mr-0.1-lr-1/model-fin-model.pkl',
    './imagenett-models/resnet50-mr-0.2-backup-lr-1/model-fin-model.pkl',
    './imagenett-models/resnet50-mr-0.3-lr1/model-fin-model.pkl',
    './imagenett-models/resnet50-mr-0.5-lr1/model-fin-model.pkl',
    
    './imagenett-models/resnet50-sam-0.2-lr-1/model-fin-model.pkl',
    './imagenett-models/resnet50-sam-0.5-lr1/model-fin-model.pkl',
    './imagenett-models/resnet50-sam-1-lr1/model-fin-model.pkl',
    './imagenett-models/resnet50-sam-2-lr1/model-fin-model.pkl',
    './imagenett-models/resnet50-sam-3-lr-1/model-fin-model.pkl',
    
    './imagenett-models/resnet50-jr-0.001-lr1/model-fin-model.pkl',
    './imagenett-models/resnet50-jr-0.01-lr1/model-fin-model.pkl',
    './imagenett-models/resnet50-jr-0.1-backup-lr-1-special/model-fin-model.pkl',
    './imagenett-models/resnet50-jr-0.5-lr1/model-fin-model.pkl',
    './imagenett-models/resnet50-jr-1-lr1/model-fin-model.pkl',
    
    './imagenett-models/resnet50-ig-0.1-lr1/model-fin-model.pkl',
    './imagenett-models/resnet50-ig-0.5-lr1/model-fin-model.pkl',
    './imagenett-models/resnet50-ig-1-lr1/model-fin-model.pkl',
    './imagenett-models/resnet50-ig-1.5-lr1/model-fin-model.pkl',
    './imagenett-models/resnet50-ig-2-lr1/model-fin-model.pkl',
]

gr_model_path1=[
    './imagenett-models/resnet50-mr-0.05-lr1-seed-1/model-fin-model.pkl',
    './imagenett-models/resnet50-mr-0.1-backup-lr-1-seed-1/model-fin-model.pkl',
    './imagenett-models/resnet50-mr-0.2-backup-lr-1-seed-1/model-fin-model.pkl',
    './imagenett-models/resnet50-mr-0.3-backup-lr-1-seed-1/model-fin-model.pkl',
    './imagenett-models/resnet50-mr-0.5-backup-lr-1-seed-1/model-fin-model.pkl',
    
    './imagenett-models/resnet50-sam-0.2-lr1-seed-1/model-fin-model.pkl',
    './imagenett-models/resnet50-sam-0.5-backup-lr-1-seed-1/model-fin-model.pkl',
    './imagenett-models/resnet50-sam-1-backup-lr-1-seed-1/model-fin-model.pkl',
    './imagenett-models/resnet50-sam-2-backup-lr-1-seed-1/model-fin-model.pkl',
    './imagenett-models/resnet50-sam-3-backup-lr-1-seed-1/model-fin-model.pkl',
    
    './imagenett-models/resnet50-jr-0.001-backup-lr-1-seed-1/model-fin-model.pkl',
    './imagenett-models/resnet50-jr-0.01-backup-lr-1-seed-1/model-fin-model.pkl',
    './imagenett-models/resnet50-jr-0.1-backup-lr-1-seed-1/model-fin-model.pkl',
    './imagenett-models/resnet50-jr-0.5-backup-lr-1-seed-1/model-fin-model.pkl',
    './imagenett-models/resnet50-jr-1-backup-lr-1-seed-1/model-fin-model.pkl',
    
    './imagenett-models/resnet50-ig-0.1-backup-lr-1-seed-1/model-fin-model.pkl',
    './imagenett-models/resnet50-ig-0.5-backup-lr-1-seed-1/model-fin-model.pkl',
    './imagenett-models/resnet50-ig-1-backup-lr-1-seed-1/model-fin-model.pkl',
    './imagenett-models/resnet50-ig-1.5-backup-lr-1-seed-1/model-fin-model.pkl',
    './imagenett-models/resnet50-ig-2-backup-lr-1-seed-1/model-fin-model.pkl',
]

gr_model_path2=[
    './imagenett-models/resnet50-mr-0.05-lr1-seed-2/model-fin-model.pkl',
    './imagenett-models/resnet50-mr-0.1-backup-lr-1-seed-2/model-fin-model.pkl',
    './imagenett-models/resnet50-mr-0.2-backup-lr-1-seed-2/model-fin-model.pkl',
    './imagenett-models/resnet50-mr-0.3-backup-lr-1-seed-2/model-fin-model.pkl',
    './imagenett-models/resnet50-mr-0.5-backup-lr-1-seed-2/model-fin-model.pkl',
    
    './imagenett-models/resnet50-sam-0.2-lr1-seed-2/model-fin-model.pkl',
    './imagenett-models/resnet50-sam-0.5-backup-lr-1-seed-2/model-fin-model.pkl',
    './imagenett-models/resnet50-sam-1-backup-lr-1-seed-2/model-fin-model.pkl',
    './imagenett-models/resnet50-sam-2-backup-lr-1-seed-2/model-fin-model.pkl',
    './imagenett-models/resnet50-sam-3-backup-lr-1-seed-2/model-fin-model.pkl',
    
    './imagenett-models/resnet50-jr-0.001-backup-lr-1-seed-2/model-fin-model.pkl',
    './imagenett-models/resnet50-jr-0.01-backup-lr-1-seed-2/model-fin-model.pkl',
    './imagenett-models/resnet50-jr-0.1-backup-lr-1-seed-2/model-fin-model.pkl',
    './imagenett-models/resnet50-jr-0.5-backup-lr-1-seed-2/model-fin-model.pkl',
    './imagenett-models/resnet50-jr-1-backup-lr-1-seed-2/model-fin-model.pkl',
    
    './imagenett-models/resnet50-ig-0.1-backup-lr-1-seed-2/model-fin-model.pkl',
    './imagenett-models/resnet50-ig-0.5-backup-lr-1-seed-2/model-fin-model.pkl',
    './imagenett-models/resnet50-ig-1-backup-lr-1-seed-2/model-fin-model.pkl',
    './imagenett-models/resnet50-ig-1.5-backup-lr-1-seed-2/model-fin-model.pkl',
    './imagenett-models/resnet50-ig-2-backup-lr-1-seed-2/model-fin-model.pkl',
]

In [ ]:
gr_seed0=[]
for path in gr_model_path0:
    tmp0=get_smoothness('resnet50', path)
    gr_seed0.append(tmp0)
data0 = pd.DataFrame(gr_seed0)
data0.to_csv('4_2_MS_gr_seed0.csv')

gr_seed1=[]
for path in gr_model_path1:
    tmp1=get_smoothness('resnet50', path)
    gr_seed1.append(tmp1)
data1 = pd.DataFrame(gr_seed1)
data1.to_csv('4_2_MS_gr_seed1.csv')

gr_seed2=[]
for path in gr_model_path2:
    tmp2=get_smoothness('resnet50', path)
    gr_seed2.append(tmp2)
data2 = pd.DataFrame(gr_seed2)
data2.to_csv('4_2_MS_gr_seed2.csv')

In [10]:
adv_model_path=[
    './imagenett-models/resnet50-adv-0.05-backup-lr-1/model-fin-model.pkl',
    './imagenett-models/resnet50-adv-0.05-backup-lr-1-seed-1/model-fin-model.pkl',
    './imagenett-models/resnet50-adv-0.05-backup-lr-1-seed-2/model-fin-model.pkl',
    './imagenett-models/resnet50-adv-0.1-backup-lr-1/model-fin-model.pkl',
    './imagenett-models/resnet50-adv-0.1-backup-lr-1-seed-1/model-fin-model.pkl',
    './imagenett-models/resnet50-adv-0.1-backup-lr-1-seed-2/model-fin-model.pkl',
    './imagenett-models/resnet50-adv-0.2-backup-lr-1/model-fin-model.pkl',
    './imagenett-models/resnet50-adv-0.2-backup-lr-1-seed-1/model-fin-model.pkl',
    './imagenett-models/resnet50-adv-0.2-backup-lr-1-seed-2/model-fin-model.pkl',
    './imagenett-models/resnet50-adv-0.5-backup-lr-1/model-fin-model.pkl',
    './imagenett-models/resnet50-adv-0.5-backup-lr-1-seed-1/model-fin-model.pkl',
    './imagenett-models/resnet50-adv-0.5-backup-lr-1-seed-2/model-fin-model.pkl',
    './imagenett-models/resnet50-adv-1-backup-lr-1/model-fin-model.pkl',
    './imagenett-models/resnet50-adv-1-backup-lr-1-seed-1/model-fin-model.pkl',
    './imagenett-models/resnet50-adv-1-backup-lr-1-seed-2/model-fin-model.pkl',
    './imagenett-models/resnet50-adv-3-backup-lr-1/model-fin-model.pkl',
    './imagenett-models/resnet50-adv-3-backup-lr-1-seed-1/model-fin-model.pkl',
    './imagenett-models/resnet50-adv-3-backup-lr-1-seed-2/model-fin-model.pkl',
    './imagenett-models/resnet50-adv-5-backup-lr-1/model-fin-model.pkl',
    './imagenett-models/resnet50-adv-5-backup-lr-1-seed-1/model-fin-model.pkl',
    './imagenett-models/resnet50-adv-5-backup-lr-1-seed-2/model-fin-model.pkl',
]

In [11]:
adv_seed=[]
for path in adv_model_path:
    tmp0=get_smoothness('resnet50', path)
    adv_seed.append(tmp0)
data0 = pd.DataFrame(adv_seed)
data0.to_csv('4_2_MS_adv_seed.csv')

sample smoothness: 0.6503782876700961
sample smoothness: 1.5592437260746572
sample smoothness: 1.043912480139304
sample smoothness: 0.46822292030826906
sample smoothness: 1.2720464730854992
sample smoothness: 0.703806132302848
sample smoothness: 0.25869424538896346
sample smoothness: 0.3620293409003697
sample smoothness: 0.39720622208270684
sample smoothness: 0.1196108070723226
sample smoothness: 0.1706910195490073
sample smoothness: 0.28021495853656747
sample smoothness: 0.13568588635543
sample smoothness: 0.12839021780141957
sample smoothness: 0.12238246306456581
sample smoothness: 0.06290230980918977
sample smoothness: 0.04955118038608985
sample smoothness: 0.05001452404832151
sample smoothness: 0.0233225348511548
sample smoothness: 0.0255133855774948
sample smoothness: 0.026829441270643482


In [10]:
model_path = [
    './imagenett-models/resnet50-backup-lr-1/model-fin-model.pkl',
    './imagenett-models/resnet50-adv-0.05-backup-lr-1/model-fin-model.pkl',
    # './imagenett-models/resnet50-adv-0.05-backup-lr-1/best-robust-eval-model.pkl',
    './imagenett-models/resnet50-adv-0.05-backup-lr-1-seed-1/model-fin-model.pkl',
    # './imagenett-models/resnet50-adv-0.05-backup-lr-1-seed-1/best-robust-eval-model.pkl',
    './imagenett-models/resnet50-adv-0.05-backup-lr-1-seed-2/model-fin-model.pkl',
    # './imagenett-models/resnet50-adv-0.05-backup-lr-1-seed-2/best-robust-eval-model.pkl',
    './imagenett-models/resnet50-adv-0.1-backup-lr-1/model-fin-model.pkl',
    # './imagenett-models/resnet50-adv-0.1-backup-lr-1/best-robust-eval-model.pkl',
    './imagenett-models/resnet50-adv-0.1-backup-lr-1-seed-1/model-fin-model.pkl',
    # './imagenett-models/resnet50-adv-0.1-backup-lr-1-seed-1/best-robust-eval-model.pkl',
    './imagenett-models/resnet50-adv-0.1-backup-lr-1-seed-2/model-fin-model.pkl',
    # './imagenett-models/resnet50-adv-0.1-backup-lr-1-seed-2/best-robust-eval-model.pkl',
    './imagenett-models/resnet50-adv-0.2-backup-lr-1/model-fin-model.pkl',
    # './imagenett-models/resnet50-adv-0.2-backup-lr-1/best-robust-eval-model.pkl',
    './imagenett-models/resnet50-adv-0.2-backup-lr-1-seed-1/model-fin-model.pkl',
    # './imagenett-models/resnet50-adv-0.2-backup-lr-1-seed-1/best-robust-eval-model.pkl',
    './imagenett-models/resnet50-adv-0.2-backup-lr-1-seed-2/model-fin-model.pkl',
    # './imagenett-models/resnet50-adv-0.2-backup-lr-1-seed-2/best-robust-eval-model.pkl',
    './imagenett-models/resnet50-adv-0.5-backup-lr-1/model-fin-model.pkl',
    # './imagenett-models/resnet50-adv-0.5-backup-lr-1/best-robust-eval-model.pkl',
    './imagenett-models/resnet50-adv-0.5-backup-lr-1-seed-1/model-fin-model.pkl',
    # './imagenett-models/resnet50-adv-0.5-backup-lr-1-seed-1/best-robust-eval-model.pkl',
    './imagenett-models/resnet50-adv-0.5-backup-lr-1-seed-2/model-fin-model.pkl',
    # './imagenett-models/resnet50-adv-0.5-backup-lr-1-seed-2/best-robust-eval-model.pkl',
    './imagenett-models/resnet50-adv-1-backup-lr-1/model-fin-model.pkl',
    # './imagenett-models/resnet50-adv-1-backup-lr-1/best-robust-eval-model.pkl',
    './imagenett-models/resnet50-adv-1-backup-lr-1-seed-1/model-fin-model.pkl',
    # './imagenett-models/resnet50-adv-1-backup-lr-1-seed-1/best-robust-eval-model.pkl',
    './imagenett-models/resnet50-adv-1-backup-lr-1-seed-2/model-fin-model.pkl',
    # './imagenett-models/resnet50-adv-1-backup-lr-1-seed-2/best-robust-eval-model.pkl',
    './imagenett-models/resnet50-adv-3-backup-lr-1/model-fin-model.pkl',
    './imagenett-models/resnet50-adv-3-backup-lr-1-seed-1/model-fin-model.pkl',
    # './imagenett-models/resnet50-adv-3-backup-lr-1-seed-1/best-robust-eval-model.pkl',
    './imagenett-models/resnet50-adv-3-backup-lr-1-seed-2/model-fin-model.pkl',
    # './imagenett-models/resnet50-adv-3-backup-lr-1-seed-2/best-robust-eval-model.pkl',
    './imagenett-models/resnet50-adv-5-backup-lr-1/model-fin-model.pkl',
    # './imagenett-models/resnet50-adv-5-backup-lr-1/best-robust-eval-model.pkl'
    './imagenett-models/resnet50-adv-5-backup-lr-1-seed-1/model-fin-model.pkl',
    # './imagenett-models/resnet50-adv-5-backup-lr-1-seed-1/best-robust-eval-model.pkl',
    './imagenett-models/resnet50-adv-5-backup-lr-1-seed-2/model-fin-model.pkl',
    # './imagenett-models/resnet50-adv-5-backup-lr-1-seed-2/best-robust-eval-model.pkl',
    # './imagenett-models/resnet50-jr-sam-lr1/model-fin-model.pkl',
    # './imagenett-models/resnet50-jr-sam-lr1/best-clean-eval-model.pkl',
    # './imagenett-models/resnet50-ir-sam-lr1/model-fin-model.pkl',
    # './imagenett-models/resnet50-ir-sam-lr1/best-clean-eval-model.pkl',
    
]

model_path1 = [
#     './imagenett-models/resnet50-backup-lr-1-seed-1/model-fin-model.pkl',
      # './imagenett-models/resnet50-mr-0.05-lr-1/model-fin-model.pkl',
      # './imagenett-models/resnet50-mr-0.3-lr1/model-fin-model.pkl',
      # './imagenett-models/resnet50-mr-0.5-lr1/model-fin-model.pkl',
      # './imagenett-models/resnet50-mr-0.7-lr1/model-fin-model.pkl',
      # './imagenett-models/resnet50-mr-1-lr1/model-fin-model.pkl',
      # './imagenett-models/resnet50-sam-0.2-lr-1/model-fin-model.pkl',
      # './imagenett-models/resnet50-sam-0.5-lr1/model-fin-model.pkl',
      # './imagenett-models/resnet50-sam-1-lr1/model-fin-model.pkl',
      # './imagenett-models/resnet50-sam-2-lr1/model-fin-model.pkl',
      # './imagenett-models/resnet50-sam-3-lr-1/model-fin-model.pkl',
    './imagenett-models/resnet50-jr-0.001-lr1/model-fin-model.pkl',
#     './imagenett-models/resnet50-jr-0.001-backup-lr-1-seed-1/best-clean-eval-model.pkl',
    './imagenett-models/resnet50-jr-0.01-lr1/model-fin-model.pkl',
#     './imagenett-models/resnet50-jr-0.01-backup-lr-1-seed-1/best-clean-eval-model.pkl',
    './imagenett-models/resnet50-jr-0.1-backup-lr-1-special/model-fin-model.pkl',
#     './imagenett-models/resnet50-jr-0.1-backup-lr-1-seed-1/best-clean-eval-model.pkl',
    './imagenett-models/resnet50-jr-0.5-lr1/model-fin-model.pkl',
#     './imagenett-models/resnet50-jr-0.5-backup-lr-1-seed-1/best-clean-eval-model.pkl',
    './imagenett-models/resnet50-jr-1-lr1/model-fin-model.pkl',
    # './imagenett-models/resnet50-jr-1-backup-lr-1-seed-1/best-clean-eval-model.pkl',
    './imagenett-models/resnet50-ig-0.1-lr1/model-fin-model.pkl',
    # './imagenett-models/resnet50-ig-0.1-backup-lr-1-seed-1/best-clean-eval-model.pkl',
    './imagenett-models/resnet50-ig-0.5-lr1/model-fin-model.pkl',
    # './imagenett-models/resnet50-ig-0.5-backup-lr-1-seed-1/best-clean-eval-model.pkl',
    './imagenett-models/resnet50-ig-1-lr1/model-fin-model.pkl',
    # './imagenett-models/resnet50-ig-1-backup-lr-1-seed-1/best-clean-eval-model.pkl',
    './imagenett-models/resnet50-ig-1.5-lr1/model-fin-model.pkl',
    # './imagenett-models/resnet50-ig-1.5-backup-lr-1-seed-1/best-clean-eval-model.pkl',
    './imagenett-models/resnet50-ig-2-lr1/model-fin-model.pkl',
    # './imagenett-models/resnet50-ig-2-backup-lr-1-seed-1/best-clean-eval-model.pkl',
    
    # './imagenett-models/resnet50-cutout-80-backup-lr-1-seed-1/model-fin-model.pkl',
    # './imagenett-models/resnet50-cutout-80-backup-lr-1-seed-1/best-clean-eval-model.pkl',
    # './imagenett-models/resnet50-cutout-100-backup-lr-1-seed-1/model-fin-model.pkl',
    # './imagenett-models/resnet50-cutout-100-backup-lr-1-seed-1/best-clean-eval-model.pkl',
    # './imagenett-models/resnet50-cutout-120-backup-lr-1-seed-1/model-fin-model.pkl',
    # './imagenett-models/resnet50-cutout-120-backup-lr-1-seed-1/best-clean-eval-model.pkl',
    # './imagenett-models/resnet50-cutout-140-backup-lr-1-seed-1/model-fin-model.pkl',
    # './imagenett-models/resnet50-cutout-140-backup-lr-1-seed-1/best-clean-eval-model.pkl',
    # './imagenett-models/resnet50-cutout-160-backup-lr-1-seed-1/model-fin-model.pkl',
    # './imagenett-models/resnet50-cutout-160-backup-lr-1-seed-1/best-clean-eval-model.pkl',
    # './imagenett-models/resnet50-cutmix-0.1-backup-lr-1-seed-1/model-fin-model.pkl',
    # './imagenett-models/resnet50-cutmix-0.1-backup-lr-1-seed-1/best-clean-eval-model.pkl',
    # './imagenett-models/resnet50-cutmix-0.3-backup-lr-1-seed-1/model-fin-model.pkl',
    # './imagenett-models/resnet50-cutmix-0.3-backup-lr-1-seed-1/best-clean-eval-model.pkl',
    # './imagenett-models/resnet50-cutmix-0.5-backup-lr-1-seed-1/model-fin-model.pkl',
    # './imagenett-models/resnet50-cutmix-0.5-backup-lr-1-seed-1/best-clean-eval-model.pkl',
    # './imagenett-models/resnet50-cutmix-0.7-backup-lr-1-seed-1/model-fin-model.pkl',
    # './imagenett-models/resnet50-cutmix-0.7-backup-lr-1-seed-1/best-clean-eval-model.pkl',
    # './imagenett-models/resnet50-cutmix-0.9-backup-lr-1-seed-1/model-fin-model.pkl',
    # './imagenett-models/resnet50-cutmix-0.9-backup-lr-1-seed-1/best-clean-eval-model.pkl',
    # './imagenett-models/resnet50-mixup-0.1-backup-lr-1-seed-1/model-fin-model.pkl',
    # './imagenett-models/resnet50-mixup-0.1-backup-lr-1-seed-1/best-clean-eval-model.pkl',
    # './imagenett-models/resnet50-mixup-0.3-backup-lr-1-seed-1/model-fin-model.pkl',
    # './imagenett-models/resnet50-mixup-0.3-backup-lr-1-seed-1/best-clean-eval-model.pkl',
    # './imagenett-models/resnet50-mixup-0.5-backup-lr-1-seed-1/model-fin-model.pkl',
    # './imagenett-models/resnet50-mixup-0.5-backup-lr-1-seed-1/best-clean-eval-model.pkl',
    # './imagenett-models/resnet50-mixup-0.7-backup-lr-1-seed-1/model-fin-model.pkl',
    # './imagenett-models/resnet50-mixup-0.7-backup-lr-1-seed-1/best-clean-eval-model.pkl',
    # './imagenett-models/resnet50-mixup-0.9-backup-lr-1-seed-1/model-fin-model.pkl',
    # './imagenett-models/resnet50-mixup-0.9-backup-lr-1-seed-1/best-clean-eval-model.pkl',
    # './imagenett-models/resnet50-label-smoothing-0.1-backup-lr-1-seed-1/model-fin-model.pkl',
    # './imagenett-models/resnet50-label-smoothing-0.1-backup-lr-1-seed-1/best-clean-eval-model.pkl',
    # './imagenett-models/resnet50-label-smoothing-0.2-backup-lr-1-seed-1/model-fin-model.pkl',
    # './imagenett-models/resnet50-label-smoothing-0.2-backup-lr-1-seed-1/best-clean-eval-model.pkl',
    # './imagenett-models/resnet50-label-smoothing-0.3-backup-lr-1-seed-1/model-fin-model.pkl',
    # './imagenett-models/resnet50-label-smoothing-0.3-backup-lr-1-seed-1/best-clean-eval-model.pkl',
    # './imagenett-models/resnet50-label-smoothing-0.4-backup-lr-1-seed-1/model-fin-model.pkl',
    # './imagenett-models/resnet50-label-smoothing-0.4-backup-lr-1-seed-1/best-clean-eval-model.pkl',
    # './imagenett-models/resnet50-label-smoothing-0.5-backup-lr-1-seed-1/model-fin-model.pkl',
    # './imagenett-models/resnet50-label-smoothing-0.5-backup-lr-1-seed-1/best-clean-eval-model.pkl',
]

In [11]:
data1=[]
for path in model_path1:
    tmp=get_smoothness('resnet50', path)
    data1.append(tmp)
data = pd.DataFrame(data1)
data.to_csv('MS_GR_seed0_Imagenette.csv')
# get_smoothness('resnet50','./imagenett-models/resnet50-mixup-0.1/model-fin-model.pkl' )
    

sample smoothness: 2.2480777839475783
sample smoothness: 1.197601296257397
sample smoothness: 0.34296571369830403
sample smoothness: 0.20478526359667312
sample smoothness: 0.17092204818062784
sample smoothness: 0.9509919900209313
sample smoothness: 0.3870185034081554
sample smoothness: 0.23528495026138402
sample smoothness: 0.21883400685280774
sample smoothness: 0.16323668377774417


In [ ]:
sample smoothness: 2.313636848509407
sample smoothness: 1.5810724651169636
sample smoothness: 1.7937784187358006
sample smoothness: 2.386714098569214
sample smoothness: 1.0696925715627743
sample smoothness: 1.4744444806021257
sample smoothness: 1.0445889757977556
sample smoothness: 0.7060543460490044
sample smoothness: 0.5984944463949432
sample smoothness: 0.600112708025099

In [11]:
for path in model_path1:
    get_smoothness('resnet50', path)
# get_smoothness('resnet50','./imagenett-models/resnet50-mixup-0.1/model-fin-model.pkl' )

sample smoothness: 88.11085052013397
sample smoothness: 99.40580779671669
sample smoothness: 99.40580563545227
sample smoothness: 166.51654475212098
sample smoothness: 162.95472141027452
sample smoothness: 143.28165313005448
sample smoothness: 143.03421003222465
sample smoothness: 94.27692071259021
sample smoothness: 90.17191184282302
sample smoothness: 75.69747719287872
sample smoothness: 75.6974720543623
sample smoothness: 155.7631144952774
sample smoothness: 154.2839228606224
sample smoothness: 495.4215804481506
sample smoothness: 476.57598279476167
sample smoothness: 786.4516675949096
sample smoothness: 786.4516665649414
sample smoothness: 1040.811368818283
sample smoothness: 556.9150151634217
sample smoothness: 1024.942768764496
sample smoothness: 1024.9426476097108
sample smoothness: 89.77579149007798
sample smoothness: 89.77578657388688
sample smoothness: 211.93423554182053
sample smoothness: 211.93422718048095
sample smoothness: 251.25670306682588
sample smoothness: 251.2567144

In [32]:
gr_model_path = [
    # './imagenett-models/resnet50-mr-0.005-lr-1/model-fin-model.pkl',
    # './imagenett-models/resnet50-mr-0.005-lr-1/best-clean-eval-model.pkl',
    # './imagenett-models/resnet50-mr-0.01-lr-1/model-fin-model.pkl',
    # './imagenett-models/resnet50-mr-0.01-lr-1/best-clean-eval-model.pkl',
    # './imagenett-models/resnet50-mr-0.05-lr-1/model-fin-model.pkl',
    # './imagenett-models/resnet50-mr-0.05-lr-1/best-clean-eval-model.pkl',
    # './imagenett-models/resnet50-mr-0.1-lr-1/model-fin-model.pkl',
    # './imagenett-models/resnet50-mr-0.1-lr-1/best-clean-eval-model.pkl',
    './imagenett-models/resnet50-mr-0.3-lr1/model-fin-model.pkl',
    './imagenett-models/resnet50-mr-0.3-lr1/best-clean-eval-model.pkl',
    './imagenett-models/resnet50-mr-0.7-lr1/model-fin-model.pkl',
    './imagenett-models/resnet50-mr-0.7-lr1/best-clean-eval-model.pkl',
    # './imagenett-models/resnet50-mr-0.5-lr1/model-fin-model.pkl',
    # './imagenett-models/resnet50-mr-0.5-lr1/best-clean-eval-model.pkl',
    # './imagenett-models/resnet50-mr-1-lr1/model-fin-model.pkl',
    # './imagenett-models/resnet50-mr-1-lr1/best-clean-eval-model.pkl',
    # './imagenett-models/resnet50-mr-1.5-lr1/model-fin-model.pkl',
    # './imagenett-models/resnet50-mr-1.5-lr1/best-clean-eval-model.pkl',
    # './imagenett-models/resnet50-sam-0.2-lr-1/model-fin-model.pkl',
    # './imagenett-models/resnet50-sam-0.2-lr-1/best-clean-eval-model.pkl',
    # './imagenett-models/resnet50-sam-0.5-lr1/model-fin-model.pkl',
    # './imagenett-models/resnet50-sam-0.5-lr1/best-clean-eval-model.pkl',
    # './imagenett-models/resnet50-sam-1-lr1/model-fin-model.pkl',
    # './imagenett-models/resnet50-sam-1-lr1/best-clean-eval-model.pkl',
    # './imagenett-models/resnet50-sam-2-lr1/model-fin-model.pkl',
    # './imagenett-models/resnet50-sam-2-lr1/best-clean-eval-model.pkl',
    # './imagenett-models/resnet50-sam-3-lr-1/model-fin-model.pkl',
    # './imagenett-models/resnet50-sam-3-lr-1/best-clean-eval-model.pkl',
    # './imagenett-models/resnet50-jr-0.00001-lr1/model-fin-model.pkl',
    # './imagenett-models/resnet50-jr-0.00001-lr1/best-clean-eval-model.pkl',
    # './imagenett-models/resnet50-jr-0.0001-lr1/model-fin-model.pkl',
    # './imagenett-models/resnet50-jr-0.0001-lr1/best-clean-eval-model.pkl',
    # './imagenett-models/resnet50-jr-0.001-lr1/model-fin-model.pkl',
    # './imagenett-models/resnet50-jr-0.001-lr1/best-clean-eval-model.pkl',
    # './imagenett-models/resnet50-jr-0.01-lr1/model-fin-model.pkl',
    # './imagenett-models/resnet50-jr-0.01-lr1/best-clean-eval-model.pkl',
    # './imagenett-models/resnet50-jr-0.1-lr1/model-fin-model.pkl',
    # './imagenett-models/resnet50-jr-0.1-lr1/best-clean-eval-model.pkl',
    # './imagenett-models/resnet50-jr-0.5-lr1/model-fin-model.pkl',
    # './imagenett-models/resnet50-jr-0.5-lr1/best-clean-eval-model.pkl',
    # './imagenett-models/resnet50-jr-1-lr1/model-fin-model.pkl',
    # './imagenett-models/resnet50-jr-1-lr1/best-clean-eval-model.pkl',
    # './imagenett-models/resnet50-ig-0.1-lr1/model-fin-model.pkl',
    # './imagenett-models/resnet50-ig-0.1-lr1/best-clean-eval-model.pkl',
    # './imagenett-models/resnet50-ig-0.5-lr1/model-fin-model.pkl',
    # './imagenett-models/resnet50-ig-0.5-lr1/best-clean-eval-model.pkl',
    # './imagenett-models/resnet50-ig-1-lr1/model-fin-model.pkl',
    # './imagenett-models/resnet50-ig-1-lr1/best-clean-eval-model.pkl',
    # './imagenett-models/resnet50-ig-1.5-lr1/model-fin-model.pkl',
    # './imagenett-models/resnet50-ig-1.5-lr1/best-clean-eval-model.pkl',
    # './imagenett-models/resnet50-ig-2-lr1/model-fin-model.pkl',
    # './imagenett-models/resnet50-ig-2-lr1/best-clean-eval-model.pkl',
    # './imagenett-models/resnet50-label-smoothing-0.4-backup-lr-1-seed-1/model-fin-model.pkl',
    # './imagenett-models/resnet50-label-smoothing-0.4-backup-lr-1-seed-1/best-clean-eval-model.pkl',
    # './imagenett-models/resnet50-label-smoothing-0.4-backup-lr-1-seed-2/model-fin-model.pkl',
    # './imagenett-models/resnet50-label-smoothing-0.4-backup-lr-1-seed-2/best-clean-eval-model.pkl',
]

In [19]:
for path in gr_model_path:
    get_smoothness('resnet50', path)

sample smoothness: 51.74137952625752
sample smoothness: 52.80206298559904
sample smoothness: 44.8036814147234
sample smoothness: 40.5184642034769
sample smoothness: 44.64878025919199
sample smoothness: 44.64879325479269
sample smoothness: 36.02784828543663
sample smoothness: 34.010929448604585


In [15]:
for path in gr_model_path:
    get_smoothness('resnet50', path)

sample smoothness: 122.06869396209717
sample smoothness: 89.43435546517372
sample smoothness: 293.5933650171757
sample smoothness: 203.66464141726493
sample smoothness: 94.21806448996067
sample smoothness: 86.68504078388214
sample smoothness: 17.769461377859116
sample smoothness: 15.25690479248762
sample smoothness: 78.68719763934612
sample smoothness: 68.78698062300683
sample smoothness: 43.44151239275932
sample smoothness: 42.8151385396719
sample smoothness: 219.4116647195816
sample smoothness: 219.41170211791993
sample smoothness: 121.72864272475243
sample smoothness: 122.7431968832016
sample smoothness: 89.88959269165993
sample smoothness: 81.51120047330856
sample smoothness: 23.7117497164011
sample smoothness: 23.71175237596035
sample smoothness: 6.705898668169976
sample smoothness: 6.6937747293710705
sample smoothness: 3.979092601314187
sample smoothness: 4.4532671061903235
sample smoothness: 2.8653144648298623
sample smoothness: 2.8653147226944564
sample smoothness: 15.938231879

In [33]:
for path in gr_model_path:
    get_smoothness('resnet50', path)

sample smoothness: 47.27090946555138
sample smoothness: 60.654480469822886
sample smoothness: 18.15461844176054
sample smoothness: 18.154619290828705
